<b style="font-size:30px"> Respirometry experiment </b>

# Import modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load data

In [ ]:
df = pd.read_excel('../data/Respirometric tests xls/Respirometry OUR heterotrophic growth blank.xlsx', 
                   sheet_name='Original data',
                   usecols='A,B,D,I',
#                  index_col=0,
                   skiprows=8)

In [ ]:
df.head()

In [ ]:
df.columns

# adjust and refurbish your dataframe

In [ ]:
df[' Time'] = df[' Time'] / 60 # to hours

In [ ]:
df = df.set_index(' Time')

In [ ]:
df['OUR'].plot()

In [ ]:
df['OUR'].plot(marker='o')

In [ ]:
df['OUR'][12:17.1].plot(marker='o')

# calculate your parameters mumax and bh

## log OUR

In [ ]:
logOUR = np.log10(df['OUR'][12:16].dropna())

In [ ]:
logOUR.plot(marker='o');

<div class="alert alert-success">
    <b> EXERCISE: </b> Do you remember how to use a rolling mean? Go! And plot it
</div>

In [ ]:
#.rolling().mean()... try composing this

In [ ]:
logOUR.dropna().rolling

In [ ]:
m, q = np.polyfit(np.array(logOUR.dropna().index), logOUR.dropna().values, 1)

In [ ]:
print(m, q)

In [ ]:
plt.plot(logOUR.dropna().index, logOUR.dropna().values, 'o')
plt.plot(logOUR.dropna().index, a*logOUR.dropna().index + b, '-');

$$ \mu_{max} - b_H $$

In [ ]:
m * 24

In [ ]:
mumax = m*24 - 0.62
mumax

# with statsmodels

source https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9

In [ ]:
import statsmodels.api as sm

X = np.array(logOUR.dropna().index)
y = np.array(logOUR.dropna().values)

X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model

# Note the difference in argument order
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

<div class="alert alert-success">
    <b> EXERCISE: </b> Complete the cell below and calculate the OUR yourself. Compare it with the one already present in the data
</div>

$$ OUR = k_La . (C_{sat} - C_t) - \frac{dC}{dt} $$

In [ ]:
kla = 4. # 1/h
Csat = 10. # mg/L

In [ ]:
df['newOUR'] = kla*(Csat - df['DO Probe']) - 